<a href="https://colab.research.google.com/github/mohammed21kamall/Graduation-Project/blob/main/PartOfPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import joblib

In [10]:
# Define the prediction function using the provided equation
def predict_rating(ru, rv, rvi, Suv, degree_v, n_nearest_users, cu, cv):

    sum_weights = 0
    sum_rating_diff_weighted = 0

    for v in range(1, n_nearest_users + 1):
        sum_rating_diff_weighted += (rvi - rv) * Suv * w(cu, cv)  # Numerator
        sum_weights += (degree_v + 1) * abs(Suv)  # Denominator

    if sum_weights == 0:
        return 0  # Avoid division by zero

    predicted_rating = ru + (sum_rating_diff_weighted / sum_weights)
    return predicted_rating


In [127]:
# Function to get all movie IDs rated by a specific user
def get_rated_movie_ids(user_id):
    # Filter ratings DataFrame for the specified user ID
    user_ratings = ratings[ratings['userId'] == user_id]

    # Extract and return the unique movie IDs rated by the user
    rated_movie_ids = user_ratings['movieId'].unique()

    return rated_movie_ids

In [113]:
# Function to get all movie IDs rated by users in a list
def get_rated_movie_ids_by_users(user_ids):
    rated_movies_by_users = {}  # Dictionary to store rated movies by each user

    for user_id in user_ids:
        # Filter ratings DataFrame for the specified user ID
        user_ratings = ratings[ratings['userId'] == user_id]

        # Extract and store the unique movie IDs rated by the user
        rated_movie_ids = user_ratings['movieId'].unique()
        rated_movies_by_users[user_id] = rated_movie_ids

    return rated_movies_by_users

In [3]:
# Function to get rating for a user and movie
def get_rating(ratings, user_id, movie_id):
    rating = ratings[(ratings['userId'] == user_id) & (ratings['movieId'] == movie_id)]['rating'].values
    if len(rating) > 0:
        return rating[0]
    else:
        return None

In [70]:
# Filter similar user IDs for a specified user ID
def get_similar_user_ids(user_id, threshold=0.8):
    # Filter the DataFrame to get similarity values for the specified user ID
    similarities_to_user = Suv[user_id]

    # Filter similar user IDs with similarity >= threshold
    similar_user_ids = similarities_to_user[similarities_to_user >= threshold].index.tolist()

    # Remove the specified user ID from the list if it's present
    similar_user_ids.remove(user_id) if user_id in similar_user_ids else None

    # Increase each user ID by 1
    similar_user_ids = [(int(user) + 1) for user in similar_user_ids]

    return similar_user_ids


In [31]:
# Function to get similarity values for a given user ID
def get_similarity_values(user_id, similarity_df):
    similarity_values = {}  # Dictionary to store similarity values for each user

    if user_id not in similarity_df.columns:
        return "User ID not found"  # Return a message if the user ID is not found

    for column_user_id in similarity_df.columns:
        similarity_value = similarity_df.loc[user_id, column_user_id]
        similarity_values[column_user_id] = similarity_value

    return similarity_values

In [136]:
# Function to get all movie IDs rated by users in a list
def get_list_rated_movie_ids(user_ids):
    rated_movies_by_users = {}  # Dictionary to store rated movies by each user

    for user_id in user_ids:
        # Filter ratings DataFrame for the specified user ID
        user_ratings = ratings[ratings['userId'] == user_id]

        # Extract and store the unique movie IDs rated by the user
        rated_movie_ids = user_ratings['movieId'].unique()
        rated_movies_by_users[user_id] = rated_movie_ids

    return rated_movies_by_users

The Similarity (Suv)

In [45]:
# Similarity
model_filename = 'drive/MyDrive/Dataset2/SimilarityOfUsers.csv'
Suv = pd.read_csv(model_filename)

Suv = Suv.clip(upper=1.0)
Suv = Suv.applymap(lambda x: max(0, x))
Suv = Suv.applymap(lambda x: x if x >= 0.4 else float(0.0))

Suv.head()

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,1.000000,0.0,0.0,0.516196,0.0,0.0,0.540215,0.0,0.0,0.0,...,0.0,0.480139,0.651735,0.000000,0.436931,0.00000,1.0,1.00000,0.000000,0.410598
1,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
2,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
3,0.516196,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,1.000000,0.000000,0.000000,0.88533,0.0,0.47253,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,1.0,1.0,0.000000,1.0,0.0,0.0,...,0.0,1.000000,0.000000,0.724984,0.000000,0.00000,0.0,0.00000,0.526469,0.000000


In [148]:
# Prompt the user to enter the user ID
user_id = input("Enter the user ID: ")

# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.columns:
    print("User ID not found!")
else:
    # Get similar user IDs for the specified user ID
    similar_user_ids = get_similar_user_ids(user_id)

    # Display similar user IDs
    # print("User ID:", user_id)
    print("Similar User IDs:", similar_user_ids[1:])

Enter the user ID: 1
Similar User IDs: [17, 18, 19, 27, 39, 42, 45, 57, 63, 64, 66, 68, 82, 91, 93, 96, 103, 115, 135, 160, 166, 171, 177, 182, 186, 198, 201, 202, 217, 219, 220, 226, 234, 239, 266, 274, 282, 288, 292, 294, 304, 307, 313, 328, 330, 332, 354, 368, 372, 380, 381, 385, 387, 391, 414, 425, 428, 434, 448, 452, 453, 469, 477, 480, 483, 489, 493, 514, 524, 527, 555, 561, 570, 573, 577, 580, 590, 597, 599, 600, 607, 608]


In [151]:
1# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.columns:
    print("User ID not found!")
else:

    # Prompt the user to specify the number of similar user IDs to display
    n = int(input("Enter the number of similar user IDs to display: "))

    # Check if the specified number is greater than the number of similar user IDs found
    if n > len(similar_user_ids):
        print(f"Only {len(similar_user_ids)} similar user IDs found. Displaying all.")

    # Display similar user IDs
    print("User ID:", user_id)
    n_users_from_similar_users = similar_user_ids[:n]
    List = n_users_from_similar_users[1:]
    print(f"Similar User IDs (Top {n}):", List)

Enter the number of similar user IDs to display: 5
User ID: 1
Similar User IDs (Top 5): [17, 18, 19, 27]


In [119]:
n_users_from_similar_users

[1, 17, 18]

In [85]:
# Assuming user_index1 and user_index2 are the numerical indices of the two users you want to compare
user_index1 = 1  # Example user index 1
user_index2 = 7  # Example user index 2

# Check if the user indices are within the range of the DataFrame Suv
if user_index1 >= len(Suv.index) or user_index2 >= len(Suv.index):
    print("One or both user indices are out of range!")
else:
    # Retrieve the similarity value between user_index1 and user_index2
    similarity_value = Suv.iloc[user_index1, user_index2]
    print(f"Similarity value between User {user_index1} and User {user_index2}: {similarity_value}")


Similarity value between User 1 and User 7: 0.0


In [23]:
# Function to get similarity values for a given user ID
def get_similarity_values(user_id, similarity_df):
    similarity_values = {}  # Dictionary to store similarity values for each user

    if user_id not in similarity_df.index:
        return "User ID not found"  # Return a message if the user ID is not found

    for column_user_id in similarity_df.columns:
        similarity_value = similarity_df.loc[user_id, column_user_id]
        similarity_values[column_user_id] = similarity_value

    return similarity_values

# Prompt the user to enter the user ID
user_id = input("Enter the user ID: ")

# Get similarity values for the specified user ID
similarity_values = get_similarity_values(user_id, Suv)

# Display similarity values
print("Similarity values for user", user_id, ":")
print(similarity_values)


Enter the user ID: 5
Similarity values for user 5 :
User ID not found


The Movies

Rating U (ru) & Rating V (rv)

In [8]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [92]:
# Prompt the user for input for two users and a movie
movie_id = int(input("Enter the movieId: "))
ratings_list = []

for i in range(n):
    user_id = n_users_from_similar_users[i]
    rating = get_rating(ratings, user_id, movie_id)
    ratings_list.append(rating)

# Print all ratings
print("Ratings for movie", movie_id, "are:", ratings_list)

Enter the movieId: 2554
Ratings for movie 2554 are: [None, None, None, None, None]


In [ ]:
ratings_list[1]

1.0

All Movies ID For User


In [142]:
# Example: Provide a list of user IDs
list_id = n_users_from_similar_users  # Example list of user IDs

# Get all movie IDs rated by the specified users
rated_movies_by_users = get_list_rated_movie_ids(list_id)

# Print the movie IDs rated by each user
for user_id, rated_movies in rated_movies_by_users.items():
    print(f"Movie IDs rated by User {user_id}:")
    print(rated_movies)

Movie IDs rated by User 5:
[  1  21  34  36  39  50  58 110 150 153 232 247 253 261 265 266 290 296
 300 316 318 344 349 357 364 367 380 410 457 474 475 515 527 531 534 588
 589 590 592 594 595 596 597 608]
Movie IDs rated by User 6:
[   2    3    4    5    6    7    8   10   11   13   15   16   17   19
   21   22   24   25   26   27   31   32   34   36   41   43   45   46
   47   50   54   60   61   62   65   66   76   79   86   87   88   89
   92   93   95  100  102  104  105  110  112  113  126  135  140  141
  145  146  150  151  153  158  159  160  161  163  165  168  170  171
  174  177  179  180  181  185  186  189  191  195  196  201  204  205
  207  208  209  210  212  216  217  218  219  222  224  225  230  231
  234  236  237  239  240  243  248  250  251  252  253  254  256  257
  258  261  262  265  266  267  270  271  273  274  276  277  279  281
  282  288  289  291  292  293  296  302  303  304  310  312  313  314
  315  316  317  318  327  329  330  332  333  336  337 

In [95]:
movie = pd.read_csv("drive/MyDrive/Dataset2/movies.csv")
movie = movie[['movieId', 'title']]
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
